In [45]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import torch

data = pd.read_csv("../../data/clean_weather.csv")
data = data.ffill()

In [46]:
PREDICTORS = ["tmax", "tmin", "rain"]
TARGET = "tmax_tomorrow"

scaler = StandardScaler()
data[PREDICTORS] = scaler.fit_transform(data[PREDICTORS])

split_data = np.split(data, [int(.7*len(data)), int(.85*len(data))])
(train_x, train_y), (valid_x, valid_y), (test_x, test_y) = [[torch.from_numpy(d[PREDICTORS].to_numpy()), torch.from_numpy(d[[TARGET]].to_numpy())] for d in split_data]

In [47]:
import torch

device = torch.device("cpu")

from torch.utils.data import Dataset

class WeatherDataset(Dataset):
    def __init__(self, x, y):
        self.dataset = [x,y]
        self.sequence_length = 7

    def __len__(self):
        return len(self.dataset[0]) - self.sequence_length

    def __getitem__(self, idx):
        x, y = self.dataset[0][idx:(idx+self.sequence_length)], self.dataset[1][idx:(idx+self.sequence_length)]
        return x.float(), y.float()

In [53]:
from torch.utils.data import DataLoader
from torch import nn

BATCH_SIZE = 32

train_dataset = WeatherDataset(train_x, train_y)
valid_dataset = WeatherDataset(valid_x, valid_y)
test_dataset = WeatherDataset(test_x, test_y)

train = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
valid = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=True)
test = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [54]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.rnn = nn.RNN(3, 2, 1, batch_first=True)
        self.dense = nn.Linear(2, 1)

    def forward(self, x):
        x, hidden = self.rnn(x)
        x = self.dense(x)
        return x

In [59]:
model = NeuralNetwork().to(device)
loss_fn = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=5e-5)

EPOCHS = 50
for epoch in range(EPOCHS):
    for batch, (sequence, target) in enumerate(train):
        optimizer.zero_grad()

        sequence = sequence.to(device)
        pred = model(sequence)

        loss = loss_fn(pred, target)
        loss.backward()
        optimizer.step()

    loss = loss.item()
    print(f"Epoch {epoch} train loss: {loss}")

    with torch.no_grad():
        for batch, (sequence, target) in enumerate(valid):
            sequence = sequence.to(device)
            pred = model(sequence)
            loss = loss_fn(pred, target)

    print(f"Valid loss: {loss}")

Epoch 0 train loss: 3591.80517578125
Valid loss: 3818.302734375
Epoch 1 train loss: 3234.24755859375
Valid loss: 3257.409912109375
Epoch 2 train loss: 2653.9814453125
Valid loss: 3299.129150390625
Epoch 3 train loss: 2094.543701171875
Valid loss: 2240.60107421875
Epoch 4 train loss: 1461.6103515625
Valid loss: 1982.0947265625
Epoch 5 train loss: 1719.60791015625
Valid loss: 1502.0980224609375
Epoch 6 train loss: 1230.162841796875
Valid loss: 1624.6646728515625
Epoch 7 train loss: 1304.4542236328125
Valid loss: 1281.7115478515625
Epoch 8 train loss: 901.5313720703125
Valid loss: 1145.889892578125
Epoch 9 train loss: 886.4434814453125
Valid loss: 641.3526611328125
Epoch 10 train loss: 566.681396484375
Valid loss: 737.8563232421875
Epoch 11 train loss: 529.9597778320312
Valid loss: 565.46630859375
Epoch 12 train loss: 468.3219299316406
Valid loss: 356.1773376464844
Epoch 13 train loss: 378.99072265625
Valid loss: 271.2882995605469
Epoch 14 train loss: 270.22039794921875
Valid loss: 219.75

In [63]:
pred

tensor([[[64.2513],
         [64.5311],
         [64.5296],
         [64.5305],
         [64.5315],
         [64.5336],
         [64.5341]],

        [[64.3253],
         [64.5336],
         [64.5314],
         [64.5344],
         [64.5364],
         [64.5339],
         [64.5276]],

        [[63.9540],
         [64.5034],
         [64.5137],
         [64.5210],
         [64.5210],
         [64.5220],
         [64.5151]]])